In [ ]:
import timeit

import numpy as np
import scipy.linalg as spla
import matplotlib.pyplot as plt

from pymor.algorithms.gram_schmidt import gram_schmidt
from pymor.algorithms.svd_va import mos, qr_svd
from pymor.vectorarrays.numpy import NumpyVectorSpace

In [ ]:
ns = np.logspace(3, 4, 10, dtype=int)
ms = np.linspace(10, 100, 10, dtype=int)
N, M = np.meshgrid(ns, ms)

def find_time(n, m, svd):
    random = np.random.RandomState(0)
    U = spla.qr(random.standard_normal((n, m)), mode='economic')[0]
    s = np.logspace(-20, 1, m)[::-1]
    Vh = spla.qr(random.standard_normal((m, m)), mode='economic')[0]
    A = U * s @ Vh.T
    Ava = NumpyVectorSpace(n).from_numpy(A.T)
    return timeit.timeit('svd(Ava, check=False)', number=10, globals=locals())

mos_time = np.vectorize(lambda n, m: find_time(n, m, mos))(N, M)
qr_time = np.vectorize(lambda n, m: find_time(n, m, qr_svd))(N, M)

In [ ]:
mos_time

In [ ]:
qr_time

In [ ]:
scale = qr_time / mos_time
scale

In [ ]:
scale.min(), scale.max()

In [ ]:
np.quantile(scale, 0.25), np.quantile(scale, 0.75)

In [ ]:
fig, ax = plt.subplots()
cs = ax.imshow(scale)
ax.set_xlabel('m')
ax.set_ylabel('n')
ax.set_xticklabels([0] + ms.tolist())
ax.set_yticklabels([0] + ns.tolist())
fig.colorbar(cs)
plt.show()